### import pandas as pd
### 시각화를 위한 라이브러리
# - 파이썬 시각화에서 가장 기본적인 시각화 라이브러리
import matplotlib.pyplot as plt

### 한글처리
from matplotlib import font_manager, rc

### 운영체제 확인 라이브러리
import platform

### 시각화 시 마이너스(-, 음수) 기호 깨짐 방지
plt.rcParams["axes.unicode_minus"] = False

### OS별 한글처리
# - 윈도우 운영체게
if platform.system() == "Windows" :
    # path = "c:/Windows/Fonts/malgun.ttf"
    # font_name = font_manager.FontProperties(fname=path).get_name()
    # rc("font", family = font_name)
    
    ### 또는 아래처럼 한줄로도 가능 (아래 한글처리를 주로 사용합니다.)
    plt.rc("font", family = "Malgun Gothic")

# - Mac 운영체제
elif platform.system() == "Darwin" :
    rc("font", family = "AppleGothic")
    
else :
    print("넌 누구?")

In [2]:
cn7 = pd.read_csv("./data/labeled_data.csv")
cn7 = cn7.drop(columns=["_id", "TimeStamp","PART_FACT_PLAN_DATE", "PART_FACT_SERIAL",
                        "EQUIP_NAME", "EQUIP_CD", "Reason", "Clamp_Open_Position",
                        "Mold_Temperature_1","Mold_Temperature_2","Mold_Temperature_5","Mold_Temperature_6",
                        "Mold_Temperature_7","Mold_Temperature_8","Mold_Temperature_9","Mold_Temperature_10",
                        "Mold_Temperature_11","Mold_Temperature_12","Barrel_Temperature_7","Switch_Over_Position"])
cn7 = cn7[(cn7["PART_NAME"] =="CN7 W/S SIDE MLD'G RH") | (cn7["PART_NAME"] =="CN7 W/S SIDE MLD'G LH")]
cn7 = cn7.drop(columns=["PART_NAME"])
cn7 = cn7.replace("Y", 1).replace("N", 0)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9928\3887624601.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cn7 = cn7.replace("Y", 1).replace("N", 0)


In [3]:
def remove_outliers(df, column):
    # 1사분위수(Q1)와 3사분위수(Q3) 계산
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    
    # IQR 계산
    IQR = Q3 - Q1
    
    # 이상치 범위 설정
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # 이상치 제거
    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    return filtered_df

In [4]:
cn7_pass = cn7[cn7["PassOrFail"] == 1]
cn7_fail = cn7[cn7["PassOrFail"] == 0]
cn7_pass_removed = cn7_pass.copy()
cn7_pass_removed = remove_outliers(cn7_pass_removed, 'Injection_Time')
cn7_pass_removed = remove_outliers(cn7_pass_removed, 'Plasticizing_Time')
cn7_pass_removed = remove_outliers(cn7_pass_removed, 'Cycle_Time')
cn7_removed = pd.concat([cn7_pass_removed, cn7_fail], ignore_index=True)


In [5]:
cn7_removed["PassOrFail"].value_counts()

PassOrFail
1    6597
0      39
Name: count, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split


grid_df = pd.DataFrame()

# 독립변수 종속변수 분리 
y = cn7_removed["PassOrFail"]
X = cn7_removed.drop(columns=["PassOrFail"])

# 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
X_train

,Injection_Time,Filling_Time,Plasticizing_Time,Cycle_Time,Clamp_Close_Time,Cushion_Position,Plasticizing_Position,Max_Injection_Speed,Max_Screw_RPM,Average_Screw_RPM,...,Average_Back_Pressure,Barrel_Temperature_1,Barrel_Temperature_2,Barrel_Temperature_3,Barrel_Temperature_4,Barrel_Temperature_5,Barrel_Temperature_6,Hopper_Temperature,Mold_Temperature_3,Mold_Temperature_4
2619,9.53,4.41,16.799999,59.520000,7.13,653.429993,68.250000,55.799999,30.500000,29.200001,...,59.400002,276.200012,275.700012,274.899994,270.500000,255.300003,230.100006,68.000000,21.799999,23.000000
3194,9.60,4.48,16.629999,59.520000,7.11,653.419983,68.330002,55.000000,30.799999,29.200001,...,59.900002,276.200012,274.799988,274.600006,269.600006,255.199997,230.199997,65.199997,22.400000,24.100000
1564,9.58,4.47,16.870001,59.520000,7.12,653.450012,68.580002,55.599998,30.799999,292.500000,...,59.299999,275.899994,275.000000,274.500000,270.100006,254.899994,230.000000,66.900002,22.700001,24.100000
4035,9.62,4.51,16.780001,59.560001,7.12,653.460022,68.589996,55.099998,30.600000,292.399994,...,59.700001,276.500000,275.600006,275.299988,271.399994,255.000000,229.899994,66.500000,21.400000,22.600000
5112,9.51,4.39,16.790001,59.459999,7.12,653.429993,68.250000,56.099998,30.500000,29.200001,...,59.299999,275.700012,275.700012,275.500000,270.700012,255.100006,230.000000,66.500000,21.299999,22.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,9.59,4.48,16.610001,59.520000,7.11,653.429993,68.339996,55.000000,30.600000,29.200001,...,59.900002,276.000000,275.700012,275.600006,271.100006,255.199997,230.000000,64.900002,21.000000,22.200001
5191,9.54,4.42,16.700001,59.520000,7.13,653.450012,68.269997,55.700001,30.500000,29.200001,...,59.400002,276.100006,275.299988,275.299988,270.700012,254.899994,229.600006,66.800003,21.500000,22.600000
5226,9.51,4.40,16.820000,59.480000,7.13,653.429993,68.239998,56.000000,30.500000,29.200001,...,59.099998,276.500000,275.500000,275.399994,269.399994,255.100006,230.199997,67.099998,21.799999,22.900000
5390,9.51,4.40,16.860001,59.459999,7.12,653.440002,68.260002,56.000000,30.400000,29.200001,...,59.200001,276.299988,275.700012,274.899994,269.399994,255.100006,229.899994,66.099998,21.600000,22.700001


In [8]:
from sklearn.utils import resample

Xy_train = X_train.copy()
Xy_train["PassOrFail"] = y_train

# Average_Screw_RPM 기준으로 up만 추출하기
Xy_train_negative = Xy_train[Xy_train["PassOrFail"] == 0]
Xy_train_positive = Xy_train[Xy_train["PassOrFail"] == 1]

# 목표 불량 데이터 개수 (양품 데이터의 70% 수준으로 설정)
target_defective_count = int(len(X_train) * 0.7)

# 랜덤 오버샘플링 수행 (불량 데이터를 증강)
oversampled_negative_data = resample(
    Xy_train_negative,
    replace=True,
    n_samples=target_defective_count,  # 목표 불량 데이터 개수로 증강
    random_state=42
)

# 증강된 불량 데이터와 기존 양품 데이터를 병합
oversampled_data = pd.concat([Xy_train_positive, oversampled_negative_data])

# 증강 후 데이터 분리
X_resampled = oversampled_data.drop(columns=["PassOrFail"])
y_resampled = oversampled_data["PassOrFail"]

# 검증 데이터 분리
X_train, X_val, y_train, y_val  = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [9]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier


# 데이터 정규화
ss=StandardScaler()
ss.fit(X_train)
X_train_scaled = ss.transform(X_train)
X_val_scaled = ss.transform(X_val)
X_test_scaled=ss.transform(X_test)

# 모델 생성
rf = RandomForestClassifier(random_state=42)
et = ExtraTreesClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)
hgb = HistGradientBoostingClassifier(random_state=42)
xgb = XGBClassifier(random_state=42)
model_list = [rf, et, gb, hgb, xgb]

# GridSearchCV 이용해서 모든 모델 하이퍼파라미터 튜닝
for model in model_list:
    model_name = model.__class__.__name__
    gridParams = dict()
    if model_name =="XGBClassifier" :
        gridParams["n_estimators"] = [20,50]
        gridParams["max_depth"] = [10,20]
        gridParams["min_child_weight"] = [1, 2,4]
    elif model_name == "HistGradientBoostingClassifier" :
        gridParams["max_iter"] = [20, 50]
        gridParams["max_depth"] = [10, 50]
        gridParams["min_samples_leaf"] = [1,2,4]
    else :
        gridParams["n_estimators"] = [20, 50]
        gridParams["max_depth"] = [10,20]
        gridParams["min_samples_split"] = [2,5,10]
        gridParams["min_samples_leaf"] = [1,2,4]

    grid_search_model = GridSearchCV(model, gridParams, scoring='f1', cv=5, n_jobs=-1)
    grid_search_model.fit(X_train_scaled, y_train)
    model = grid_search_model.best_estimator_
    train_pred = model.predict(X_train_scaled)
    val_pred = model.predict(X_val_scaled)
    
    train_acc = accuracy_score(y_train, train_pred)
    val_acc = accuracy_score(y_val, val_pred)
    
    pre = precision_score(y_val, val_pred)
    rec = recall_score(y_val, val_pred)
    f1 = f1_score(y_val, val_pred)
    
    pred = model.predict(X_test_scaled)
    cm = confusion_matrix(y_test, pred)
    TN = cm[0, 0]  # True Negative
    FP = cm[0, 1]  # False Positive
    FN = cm[1, 0]  # False Negative
    TP = cm[1, 1]  # True Positive
    
    df_temp = pd.DataFrame([[model_name,  train_acc, val_acc, pre, rec, f1, train_acc-val_acc, TN, FP, FN, TP, str(grid_search_model.best_params_)]],
                             columns=["모델명", "훈련_정확도", "검증_정확도", "정밀도", "재현율", "f1-score", "훈련-검증", "TN", "FP", "FN", "TP", "파라미터"])

    grid_df = pd.concat([grid_df, df_temp], ignore_index=True)

grid_df

,모델명,훈련_정확도,검증_정확도,정밀도,재현율,f1-score,훈련-검증,TN,FP,FN,TP,파라미터
0,RandomForestClassifier,0.999456,0.999153,1.0,0.998562,0.999281,0.000302,11,5,5,1970,"{'max_depth': 10, 'min_samples_leaf': 2, 'min_..."
1,ExtraTreesClassifier,0.999637,0.999153,1.0,0.998562,0.999281,0.000484,11,5,4,1971,"{'max_depth': 20, 'min_samples_leaf': 1, 'min_..."
2,GradientBoostingClassifier,0.999637,0.998730,1.0,0.997843,0.998920,0.000907,11,5,5,1970,"{'max_depth': 10, 'min_samples_leaf': 2, 'min_..."
3,HistGradientBoostingClassifier,0.999637,0.997883,1.0,0.996405,0.998199,0.001754,12,4,6,1969,"{'max_depth': 10, 'max_iter': 50, 'min_samples..."
4,XGBClassifier,0.999637,0.998730,1.0,0.997843,0.998920,0.000907,11,5,5,1970,"{'max_depth': 10, 'min_child_weight': 1, 'n_es..."


In [ ]:
from sklearn.utils import resample

Xy_train = X_train.copy()
Xy_train["PassOrFail"] = y_train

# Average_Screw_RPM 기준으로 up만 추출하기
Xy_train_negative = Xy_train[Xy_train["PassOrFail"] == 0]
Xy_train_positive = Xy_train[Xy_train["PassOrFail"] == 1]

# 목표 불량 데이터 개수 (양품 데이터의 70% 수준으로 설정)
target_defective_count = int(len(X_train) * 0.7)

# 랜덤 오버샘플링 수행 (불량 데이터를 증강)
oversampled_negative_data = resample(
    Xy_train_negative,
    replace=True,
    n_samples=target_defective_count,  # 목표 불량 데이터 개수로 증강
    random_state=42
)

# 증강된 불량 데이터와 기존 양품 데이터를 병합
oversampled_data = pd.concat([Xy_train_positive, oversampled_negative_data])

# 증강 후 데이터 분리
X_resampled = oversampled_data.drop(columns=["PassOrFail"])
y_resampled = oversampled_data["PassOrFail"]

# 검증 데이터 분리
X_train, X_val, y_train, y_val  = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# ADASYN

In [17]:
# 독립변수 종속변수 분리 
y = cn7_removed["PassOrFail"]
X = cn7_removed.drop(columns=["PassOrFail"])

# 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [18]:
from imblearn.over_sampling import ADASYN
# X_train, y_train 합치기
Xy_train = X_train.copy()
Xy_train["PassOrFail"] = y_train

# Average_Screw_RPM 기준으로 up만 추출하기
Xy_train = Xy_train[Xy_train["Average_Screw_RPM"]>250]

# X_train, y_train 분리
X_train = Xy_train.drop(columns=["PassOrFail"])
y_train = Xy_train["PassOrFail"]
adasyn = ADASYN(random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)

print("증강 후")
print(X_resampled.shape, y_resampled.shape)
print("====================================")

# 검증 데이터 분리
X_train, X_val, y_train, y_val  = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

증강 후
(3358, 23) (3358,)


In [19]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier


# 데이터 정규화
ss=StandardScaler()
ss.fit(X_train)
X_train_scaled = ss.transform(X_train)
X_val_scaled = ss.transform(X_val)
X_test_scaled=ss.transform(X_test)

# 모델 생성
rf = RandomForestClassifier(random_state=42)
et = ExtraTreesClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)
hgb = HistGradientBoostingClassifier(random_state=42)
xgb = XGBClassifier(random_state=42)
model_list = [rf, et, gb, hgb, xgb]

# GridSearchCV 이용해서 모든 모델 하이퍼파라미터 튜닝
for model in model_list:
    model_name = model.__class__.__name__
    gridParams = dict()
    if model_name =="XGBClassifier" :
        gridParams["n_estimators"] = [20,50]
        gridParams["max_depth"] = [10,20]
        gridParams["min_child_weight"] = [1, 2,4]
    elif model_name == "HistGradientBoostingClassifier" :
        gridParams["max_iter"] = [20, 50]
        gridParams["max_depth"] = [10, 50]
        gridParams["min_samples_leaf"] = [1,2,4]
    else :
        gridParams["n_estimators"] = [20, 50]
        gridParams["max_depth"] = [10,20]
        gridParams["min_samples_split"] = [2,5,10]
        gridParams["min_samples_leaf"] = [1,2,4]

    grid_search_model = GridSearchCV(model, gridParams, scoring='f1', cv=5, n_jobs=-1)
    grid_search_model.fit(X_train_scaled, y_train)
    model = grid_search_model.best_estimator_
    train_pred = model.predict(X_train_scaled)
    val_pred = model.predict(X_val_scaled)
    
    train_acc = accuracy_score(y_train, train_pred)
    val_acc = accuracy_score(y_val, val_pred)
    
    pre = precision_score(y_val, val_pred)
    rec = recall_score(y_val, val_pred)
    f1 = f1_score(y_val, val_pred)
    
    pred = model.predict(X_test_scaled)
    cm = confusion_matrix(y_test, pred)
    TN = cm[0, 0]  # True Negative
    FP = cm[0, 1]  # False Positive
    FN = cm[1, 0]  # False Negative
    TP = cm[1, 1]  # True Positive
    
    df_temp = pd.DataFrame([[model_name,  train_acc, val_acc, pre, rec, f1, train_acc-val_acc, TN, FP, FN, TP, str(grid_search_model.best_params_)]],
                             columns=["모델명", "훈련_정확도", "검증_정확도", "정밀도", "재현율", "f1-score", "훈련-검증", "TN", "FP", "FN", "TP", "파라미터"])

    grid_df = pd.concat([grid_df, df_temp], ignore_index=True)

grid_df

,모델명,훈련_정확도,검증_정확도,정밀도,재현율,f1-score,훈련-검증,TN,FP,FN,TP,파라미터
0,RandomForestClassifier,0.999456,0.999153,1.000000,0.998562,0.999281,0.000302,11,5,5,1970,"{'max_depth': 10, 'min_samples_leaf': 2, 'min_..."
1,ExtraTreesClassifier,0.999637,0.999153,1.000000,0.998562,0.999281,0.000484,11,5,4,1971,"{'max_depth': 20, 'min_samples_leaf': 1, 'min_..."
2,GradientBoostingClassifier,0.999637,0.998730,1.000000,0.997843,0.998920,0.000907,11,5,5,1970,"{'max_depth': 10, 'min_samples_leaf': 2, 'min_..."
3,HistGradientBoostingClassifier,0.999637,0.997883,1.000000,0.996405,0.998199,0.001754,12,4,6,1969,"{'max_depth': 10, 'max_iter': 50, 'min_samples..."
4,XGBClassifier,0.999637,0.998730,1.000000,0.997843,0.998920,0.000907,11,5,5,1970,"{'max_depth': 10, 'min_child_weight': 1, 'n_es..."
5,RandomForestClassifier,0.999382,0.998198,0.996479,1.000000,0.998236,0.001184,7,9,3,1972,"{'max_depth': 20, 'min_samples_leaf': 2, 'min_..."
6,ExtraTreesClassifier,0.999536,0.999279,1.000000,0.998587,0.999293,0.000257,10,6,6,1969,"{'max_depth': 20, 'min_samples_leaf': 1, 'min_..."
7,GradientBoostingClassifier,0.999536,0.997838,0.996476,0.999293,0.997883,0.001699,12,4,7,1968,"{'max_depth': 10, 'min_samples_leaf': 1, 'min_..."
8,HistGradientBoostingClassifier,0.999536,0.998919,0.999293,0.998587,0.998940,0.000618,10,6,5,1970,"{'max_depth': 50, 'max_iter': 50, 'min_samples..."
9,XGBClassifier,0.999536,0.999640,0.999294,1.000000,0.999647,-0.000103,11,5,4,1971,"{'max_depth': 10, 'min_child_weight': 1, 'n_es..."


# SMOTE

In [20]:
from imblearn.over_sampling import SMOTE

# 독립변수 종속변수 분리 
y = cn7_removed["PassOrFail"]
X = cn7_removed.drop(columns=["PassOrFail"])

# 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# X_train, y_train 합치기
Xy_train = X_train.copy()
Xy_train["PassOrFail"] = y_train

# Average_Screw_RPM 기준으로 up만 추출하기
Xy_train = Xy_train[Xy_train["Average_Screw_RPM"]>250]

# X_train, y_train 분리
X_train = Xy_train.drop(columns=["PassOrFail"])
y_train = Xy_train["PassOrFail"]

# 데이터 증강
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# 검증 데이터 분리
X_train, X_val, y_train, y_val  = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [21]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier


# 데이터 정규화
ss=StandardScaler()
ss.fit(X_train)
X_train_scaled = ss.transform(X_train)
X_val_scaled = ss.transform(X_val)
X_test_scaled=ss.transform(X_test)

# 모델 생성
rf = RandomForestClassifier(random_state=42)
et = ExtraTreesClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)
hgb = HistGradientBoostingClassifier(random_state=42)
xgb = XGBClassifier(random_state=42)
model_list = [rf, et, gb, hgb, xgb]

# GridSearchCV 이용해서 모든 모델 하이퍼파라미터 튜닝
for model in model_list:
    model_name = model.__class__.__name__
    gridParams = dict()
    if model_name =="XGBClassifier" :
        gridParams["n_estimators"] = [20,50]
        gridParams["max_depth"] = [10,20]
        gridParams["min_child_weight"] = [1, 2,4]
    elif model_name == "HistGradientBoostingClassifier" :
        gridParams["max_iter"] = [20, 50]
        gridParams["max_depth"] = [10, 50]
        gridParams["min_samples_leaf"] = [1,2,4]
    else :
        gridParams["n_estimators"] = [20, 50]
        gridParams["max_depth"] = [10,20]
        gridParams["min_samples_split"] = [2,5,10]
        gridParams["min_samples_leaf"] = [1,2,4]

    grid_search_model = GridSearchCV(model, gridParams, scoring='f1', cv=5, n_jobs=-1)
    grid_search_model.fit(X_train_scaled, y_train)
    model = grid_search_model.best_estimator_
    train_pred = model.predict(X_train_scaled)
    val_pred = model.predict(X_val_scaled)
    
    train_acc = accuracy_score(y_train, train_pred)
    val_acc = accuracy_score(y_val, val_pred)
    
    pre = precision_score(y_val, val_pred)
    rec = recall_score(y_val, val_pred)
    f1 = f1_score(y_val, val_pred)
    
    pred = model.predict(X_test_scaled)
    cm = confusion_matrix(y_test, pred)
    TN = cm[0, 0]  # True Negative
    FP = cm[0, 1]  # False Positive
    FN = cm[1, 0]  # False Negative
    TP = cm[1, 1]  # True Positive
    
    df_temp = pd.DataFrame([[model_name,  train_acc, val_acc, pre, rec, f1, train_acc-val_acc, TN, FP, FN, TP, str(grid_search_model.best_params_)]],
                             columns=["모델명", "훈련_정확도", "검증_정확도", "정밀도", "재현율", "f1-score", "훈련-검증", "TN", "FP", "FN", "TP", "파라미터"])

    grid_df = pd.concat([grid_df, df_temp], ignore_index=True)

grid_df

,모델명,훈련_정확도,검증_정확도,정밀도,재현율,f1-score,훈련-검증,TN,FP,FN,TP,파라미터
0,RandomForestClassifier,0.999456,0.999153,1.000000,0.998562,0.999281,0.000302,11,5,5,1970,"{'max_depth': 10, 'min_samples_leaf': 2, 'min_..."
1,ExtraTreesClassifier,0.999637,0.999153,1.000000,0.998562,0.999281,0.000484,11,5,4,1971,"{'max_depth': 20, 'min_samples_leaf': 1, 'min_..."
2,GradientBoostingClassifier,0.999637,0.998730,1.000000,0.997843,0.998920,0.000907,11,5,5,1970,"{'max_depth': 10, 'min_samples_leaf': 2, 'min_..."
3,HistGradientBoostingClassifier,0.999637,0.997883,1.000000,0.996405,0.998199,0.001754,12,4,6,1969,"{'max_depth': 10, 'max_iter': 50, 'min_samples..."
4,XGBClassifier,0.999637,0.998730,1.000000,0.997843,0.998920,0.000907,11,5,5,1970,"{'max_depth': 10, 'min_child_weight': 1, 'n_es..."
5,RandomForestClassifier,0.999382,0.998198,0.996479,1.000000,0.998236,0.001184,7,9,3,1972,"{'max_depth': 20, 'min_samples_leaf': 2, 'min_..."
6,ExtraTreesClassifier,0.999536,0.999279,1.000000,0.998587,0.999293,0.000257,10,6,6,1969,"{'max_depth': 20, 'min_samples_leaf': 1, 'min_..."
7,GradientBoostingClassifier,0.999536,0.997838,0.996476,0.999293,0.997883,0.001699,12,4,7,1968,"{'max_depth': 10, 'min_samples_leaf': 1, 'min_..."
8,HistGradientBoostingClassifier,0.999536,0.998919,0.999293,0.998587,0.998940,0.000618,10,6,5,1970,"{'max_depth': 50, 'max_iter': 50, 'min_samples..."
9,XGBClassifier,0.999536,0.999640,0.999294,1.000000,0.999647,-0.000103,11,5,4,1971,"{'max_depth': 10, 'min_child_weight': 1, 'n_es..."
